# Kiva - preprocessed data overview

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm

tqdm.pandas()

# Import raw data

In [ ]:
ds = pd.read_parquet("../data/gen/preprocessed_2023-08-28T11-09-39_from_2019-01-01_activelender.parquet")

In [ ]:
ds = ds[(ds["tags"] != "") & (ds["tags"] != "empty")]
ds.tags = ds.tags.cat.remove_unused_categories()

In [ ]:
ds.isna().all(axis=1).sum()

In [ ]:
ds.duplicated().sum()

In [ ]:
ds.info()

## We keep only the success loans

In [ ]:
success = ds.loanAmount == ds.loanFundraisingInfo_fundedAmount
counts = success.value_counts()
counts[True] / (counts[True] + counts[False]), len(ds)

In [ ]:
# keep success only
ds = ds[success]

# Basic bipartite graphs statistic

In [ ]:
LP = ds.drop(columns="tags").drop_duplicates()

## Lender-Sector

In [ ]:
LS = ds.groupby(["lender_id", "sector_name"], observed=True).agg({"loan_shareAmount": "sum"}).reset_index()

In [ ]:
report = {}
report["kiva active LS"] = {
    "n_1": LS.lender_id.nunique(),  # number of nodes 1
    "n_2": LS.sector_name.nunique(),
    "m": len(LS),  # number of edges
    "k_1": len(LS) / LS.lender_id.nunique(),  # average degree of node 1
    "k_2": len(LS) / LS.sector_name.nunique(),  # average degree of node 2
    "k": len(LS) / (LS.lender_id.nunique() + LS.sector_name.nunique()),  # global average degree
    "density": len(LS) / (LS.lender_id.nunique() * LS.sector_name.nunique()),  # density
}

In [ ]:
LC = ds.groupby(["lender_id", "geocode_country_name"], observed=True).agg({"loan_shareAmount": "sum"}).reset_index()
LC.rename(columns={"geocode_country_name": "country_name"}, inplace=True)

In [ ]:
report["kiva active LC"] = {
    "n_1": LC.lender_id.nunique(),  # number of nodes 1
    "n_2": LC.country_name.nunique(),
    "m": len(LC),  # number of edges
    "k_1": len(LC) / LC.lender_id.nunique(),  # average degree of node 1
    "k_2": len(LC) / LC.country_name.nunique(),  # average degree of node 2
    "k": len(LC) / (LC.lender_id.nunique() + LC.country_name.nunique()),  # global average degree
    "density": len(LC) / (LC.lender_id.nunique() * LC.country_name.nunique()),  # density
}

In [ ]:
LT = ds.groupby(["lender_id", "tags"], observed=True).agg({"loan_shareAmount": "sum"}).reset_index()

In [ ]:
report["kiva active LT"] = {
    "n_1": LT.lender_id.nunique(),  # number of nodes 1
    "n_2": LT.tags.nunique(),
    "m": len(LT),  # number of edges
    "k_1": len(LT) / LT.lender_id.nunique(),  # average degree of node 1
    "k_2": len(LT) / LT.tags.nunique(),  # average degree of node 2
    "k": len(LT) / (LT.lender_id.nunique() + LT.tags.nunique()),  # global average degree
    "density": len(LT) / (LT.lender_id.nunique() * LT.tags.nunique()),  # density
}

In [ ]:
df_report = pd.DataFrame(report).T
df_report[["n_1", "n_2", "m"]] = df_report[["n_1", "n_2", "m"]].astype(int)
df_report.info()

In [ ]:
print(df_report.to_markdown(intfmt=",", floatfmt=".2f"))

# New Drawing

## Tags vs Time

In [ ]:
ds["date"] = ds.progress_apply(lambda row: list(pd.period_range(row.fundraisingDate, row.raisedDate, freq="M")), axis=1)

In [ ]:
time_ds = ds.explode("date")

## Drop some NaN

In [ ]:
ds.isna().sum()

In [ ]:
ds.dropna(subset=[COL.LOAN_AMOUNT, COL.FUNDED_AMOUNT, COL.POSTED_DATE, COL.RAISED_DATE], inplace=True)

## Collection Speed

In [ ]:
ds["funding_duration"] = ds[COL.RAISED_DATE] - ds[COL.POSTED_DATE]
ds["funding_duration_days"] = ds["funding_duration"].astype("int64") / pow(10, 9)  # seconds
ds["funding_duration_days"] = ds["funding_duration_days"] / (24 * 60 * 60)
ds[COL.SPEED] = ds[COL.FUNDED_AMOUNT] / ds["funding_duration_days"]
ds.head()

There are some projects which are fullfilled before being published.  
Let's show them and then get rid of them

In [ ]:
# some project is already fulfilled before publish
ds[ds[COL.SPEED] < 0]

In [ ]:
ds = ds[ds[COL.SPEED] >= 0]

## Encode `tags` using MultiLabelBinarizer

In [ ]:
tagdf = ds[["tags"]].to_pandas()
tagdf.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

lb = MultiLabelBinarizer()

mlb = lb.fit_transform(tagdf["tags"])
mlb.shape

In [ ]:
tags_columns = ["tag_" + i for i in lb.classes_]
tag_ds = cudf.DataFrame(mlb, columns=tags_columns, dtype="int8", index=tagdf.index)
del tagdf
tag_ds.sum()

In [ ]:
tag_ds.columns

In [ ]:
# join with the original df
ds = ds.join(tag_ds)
del tag_ds
ds.head()

In [ ]:
# sanity check here
tag_columns = [a for a in ds.columns if a.startswith("tag_")]

for testcase in range(50):
    sam = ds.sample(1)
    tags_list = sam["tags"].iloc[0]
    for atag in tag_columns:
        if atag.replace("tag_", "") in tags_list:
            assert sam[atag].iloc[0] == 1
        else:
            assert sam[atag].iloc[0] == 0

In [ ]:
# drop the orignal `tags` columns
ds.drop(["tags"], axis=1, inplace=True)

## Quickly refine `tags`

### merge 'tag_#Eco-friendly' and 'tag_#EcoFriendly'

In [ ]:
ds["tag_#EcoFriendly"] = ((ds["tag_#Eco-friendly"] + ds["tag_#EcoFriendly"]) > 0).astype("int8")
ds.drop("tag_#Eco-friendly", axis=1, inplace=True)

### Keep tags visibled to users

Take a look at this screenshot. We notice that some tags in the dataframe are not display in the website

![Alt text](images/screenshot_kiva_20230927_filter_tags.png)

These undisplayed tags are `tag_user_favorite`, `tag_user_like`, `tag_volunteer_like`, `tag_volunteer_pick`. It might because those are used internally in the kiva platform

And because those tags are not being shown to Lender, we could get rid of them here

In [ ]:
# drop some meaningless tags
ds.drop(
    ["tag_", "tag_user_favorite", "tag_user_like", "tag_volunteer_like", "tag_volunteer_pick"], axis=1, inplace=True
)

In [ ]:
ds.info()

In [ ]:
ds.isna().sum().sort_values()

# Now drawing

## Tags vs time

In [ ]:
time_df = ds[[COL.POSTED_DATE, COL.RAISED_DATE]].to_pandas()
time_df["date"] = time_df.progress_apply(
    lambda row: list(pd.date_range(row[COL.POSTED_DATE], row[COL.RAISED_DATE])), axis=1
)
time_ds = cudf.from_pandas(time_df)
del time_df
time_ds.head()

In [ ]:
time_ds = ds.merge(time_ds, left_index=True, right_index=True)
time_ds.head()

In [ ]:
time_ds = time_ds.explode("date")
time_ds["date"] = time_ds["date"].dt.floor("D")
time_ds.head()

In [ ]:
tag_columns = [a for a in ds.columns if a.startswith("tag_")]
tag_counts = []
for atag in tqdm(tag_columns):
    temp = time_ds[time_ds[atag] == 1].date.value_counts().rename(atag)
    tag_counts.append(temp)
tag_counts_concat = cudf.concat(tag_counts, axis=1)
tag_counts_concat

In [ ]:
date_range = list(pd.date_range(ds[COL.POSTED_DATE].min(), ds[COL.RAISED_DATE].max(), normalize=True))
tag_hist = cudf.DataFrame({"date": date_range})
tag_hist = tag_hist.set_index("date")
tag_hist = tag_hist.merge(tag_counts_concat, left_index=True, right_index=True, how="outer")
tag_hist_display = tag_hist.reset_index().melt(id_vars=["date"], var_name="tag")
tag_hist_display.dropna(inplace=True)
tag_hist_display = tag_hist_display.to_pandas()
tag_hist_display

In [ ]:
fig = px.histogram(
    tag_hist_display, x="date", y="value", color="tag", barmode="overlay", opacity=0.3, histnorm="percent", height=800
)
fig.show()

from above figure, we could see that some tags are only happend in a short timeframe, e.g
- `#Married`
- `Post-disbursed` ???
- `Salesforce`
- `beauty`
- `Viral`
- `MUFG`
- `reversed_crisis_support_loan`


## Number of project vs tag

In [ ]:
tag_columns = [a for a in ds.columns if a.startswith("tag_")]


class My:
    FIG_H = 800
    FIG_W = None

In [ ]:
project_count_per_tag = ds[tag_columns].sum().sort_values(ascending=False)
project_count_per_tag = project_count_per_tag.to_pandas()
project_count_per_tag.rename("number of project", inplace=True)

fig = px.bar(
    project_count_per_tag.head(20),
    orientation="v",
    text_auto=True,
    title="Number of Projects per Tag",
    height=My.FIG_H,
    width=My.FIG_W,
    labels={"x": "Categories", "y": "Number of Loans"},
)
fig.update_traces(showlegend=False)
fig.update_layout(xaxis_title=None, yaxis_title="Number of Projects")
fig.show()

## Number of project vs country

In [ ]:
proj_per_country = ds.groupby(by=["geocode.country.name"]).count()["loanAmount"].sort_values(ascending=False)
proj_per_country = proj_per_country.to_pandas()
proj_per_country.rename("number of project", inplace=True)

fig = px.bar(
    proj_per_country.head(20),
    orientation="v",
    text_auto=True,
    title="Number of Projects per Country",
    height=My.FIG_H,
    width=My.FIG_W,
    labels={"x": "Categories", "y": "Values"},
)
fig.update_traces(showlegend=False)
fig.update_layout(xaxis_title="Country", yaxis_title="Number of Projects")
fig.show()

## Collection Speed vs Tag

In [ ]:
def get_tag_performance(_df: pd.DataFrame, num_tag: int = 10) -> pd.DataFrame:
                 """get speed performance by tags, keep only first `num_tag`"""
    tags_performances = []

    for atag in tag_columns:
        mean = _df[_df[atag] == 1][COL.SPEED].mean()
        std = _df[_df[atag] == 1][COL.SPEED].std()
        count = _df[_df[atag] == 1][COL.SPEED].count()
        tags_performances.append({"tag": atag, "speed_mean": mean, "speed_std": std, "count": count})

    tags_performances = pd.DataFrame(tags_performances)
    tags_performances.dropna(subset=["speed_mean"], inplace=True)
    tags_performances.fillna(0, inplace=True)
    tags_performances.sort_values("speed_mean", inplace=True, ascending=False)
    tags_performances.set_index("tag", inplace=True)
    return tags_performances.head(num_tag)


# fig = px.bar(tags_performances, y="tag", x="mean", error_x="std", text_auto=True)
fig = px.bar(get_tag_performance(ds, 20), y="speed_mean", text_auto=True, title="Collection Speed Mean per Tag", width=My.FIG_W, height=My.FIG_H)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [ ]:
fig = px.bar(get_tag_performance(ds, 20), y="speed_mean",
text_auto=True, title="Collection Speed Mean per Tag", width=My.FIG_W, height=My.FIG_H)
fig.update_traces(te            xtfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

### Colection speed vs Tags for Countries

In [ ]:
"""convert coutry code to country name"""
code_to_name = ds[[COL.COUNTRY_NAME, COL.COUNTRY]].drop_duplicates()
code_to_name.set_index(COL.COUNTRY, inplace=True)
code_to_name = code_to_name.to_dict()[COL.COUNTRY_NAME]
assert code_to_name["VN"] == "Vietnam"

In [ ]:
country_code = "VN"
vn_df = ds[ds[COL.COUNTRY] == country_code]
fig = px.bar(
    get_tag_performance(vn_df),
    y="mean",
    text_auto=True,
    title=f"Mean Collection Speed for {code_to_name[country_code]}",
    width=My.FIG_W,
    height=My.FIG_H,
)
fig.update_traces(textfont_size=26, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [ ]:
country_code = "KE"
vn_df = ds[ds[COL.COUNTRY] == country_code]
fig = px.bar(
    get_tag_performance(vn_df),
    y="mean",
    text_auto=True,
    title=f"Mean Collection Speed for {code_to_name[country_code]}",
    width=My.FIG_W,
    height=My.FIG_H,
)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [ ]:
country_code = "KH"
vn_df = ds[ds[COL.COUNTRY] == country_code]
fig = px.bar(
    get_tag_performance(vn_df),
    y="mean",
    text_auto=True,
    title=f"Mean Collection Speed for {code_to_name[country_code]}",
    width=My.FIG_W,
    height=My.FIG_H,
)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [ ]:
country_code = "PK"
vn_df = ds[ds[COL.COUNTRY] == country_code]
fig = px.bar(
    get_tag_performance(vn_df),
    y="mean",
    text_auto=True,
    title=f"Mean Collection Speed for {code_to_name[country_code]}",
    width=My.FIG_W,
    height=My.FIG_H,
)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [ ]:
country_code = "SV"
vn_df = ds[ds[COL.COUNTRY] == country_code]
fig = px.bar(
    get_tag_performance(vn_df),
    y="mean",
    text_auto=True,
    title=f"Mean Collection Speed for {code_to_name[country_code]}",
    width=My.FIG_W,
    height=My.FIG_H,
)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

Correlation between tags and collection speed

In [ ]:
tag_df = ds[tag_columns].to_pandas()
corr = tag_df.corrwith(ds[COL.SPEED].to_pandas(), method="kendall", drop=True)
corr.rename("correlation", inplace=True)
fig = px.bar(
    corr.sort_values(ascending=False),
    orientation="h",
    text_auto=True,
    title="Correlation betwene Tags and Collection Speed",
    height=My.FIG_H,
)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(xaxis_title="kendall correlation score", yaxis_title=None)
fig.update_traces(showlegend=False)
fig.show()

### Influence of the number of tags

Above-average speed vs number of tags

In [ ]:
tag_count = ds[tag_columns].sum(axis=1)
tag_count_df = tag_count.value_counts()
tag_count_mean = tag_count.mean()
tag_count_std = tag_count.std()
fig = px.bar(
    tag_count_df.to_pandas(),
    text_auto=True,
    title="Distribution of Number of Tags per Project",
    width=My.FIG_W,
    height=My.FIG_H,
)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_xaxes(tickmode="linear")
fig.update_layout(xaxis_title="Number of Tags", yaxis_title="Number of Project")
fig.update_traces(showlegend=False)
fig.add_vline(x=tag_count_mean)
fig.show()

In [ ]:
# Number of above-average speed collection vs number of tags
speed_mean = ds[COL.SPEED].mean()
is_above_average = ds[COL.SPEED] >= speed_mean

In [ ]:
tag_count_df = tag_count.to_frame(name="tag_count").join(is_above_average.rename("is_above_average"))

In [ ]:
mn = tag_count_df.groupby("tag_count").agg(["sum", "count"])

In [ ]:
mn["percentage"] = mn["is_above_average"]["sum"] / mn["is_above_average"]["count"] * 100

In [ ]:
fig = px.bar(
    mn["percentage"],
    text_auto=True,
    title="Projects with above-average collection speed based on the number of tags",
    labels={"value": "Percentage of Project with Collection Speed above Global Average"},
    height=My.FIG_H,
)
# fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(showlegend=False)
fig.update_xaxes(tickmode="linear")
fig.show()

Effectiveness of Tags on Top 5% Collection Speed

## Pair of tags Performance

In [ ]:
from itertools import combinations

In [ ]:
tag2_columns = []
for apair in combinations(tag_columns, 2):
    col_name = "__".join(apair)
    tag2_columns.append(col_name)
    ds[col_name] = ds[apair[0]] & ds[apair[1]]
assert ds[tag2_columns].max().max() == 1
assert ds[tag2_columns].min().min() == 0

In [ ]:
speed_mean = ds[COL.SPEED].mean()
above_average = ds[ds[COL.SPEED] >= speed_mean]

In [ ]:
tag2_performances = []

for apair in tqdm(tag2_columns):
    temp = above_average[apair] * above_average[COL.SPEED]
    mean = temp.mean()
    std = temp.std()
    tag2_performances.append({"tag": apair, "mean": mean, "std": std})

In [ ]:
la = pd.DataFrame(tag2_performances)
la = la.dropna(subset=["mean"]).sort_values("mean", ascending=True)
la = la.tail(20)
la

In [ ]:
fig = px.bar(la, y="tag", x="mean", title="Effectiveness of Tags on Collection Speed", width=My.FIG_W, height=My.FIG_H)
# fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
# fig.update_xaxes(tickmode='linear')
# fig.add_vline(x=tag_count_mean)
fig.show()

# Save the results

In [ ]:
# convert to html to easiy read
!jupyter nbconvert --to html alldata_cudf.ipynb